In [8]:
import numpy as np
import pandas as pd
import chess, chess.pgn
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

from utils import fen_to_binary_features

In [9]:
# f = open()

# game_list = []

# for i in tqdm(range(10_000)):
#     game = chess.pgn.read_game(f)
#     if game is None:
#         break  # end of file

#     game_list.append(game)

In [10]:


def pgn_to_dataframe(pgn_file_path):
    """
    Parse a PGN file and create a DataFrame where each row represents a position.
    Columns in the DataFrame:
        - game_id
        - move_number
        - board_fen
        - move
        - result (e.g. '1-0', '0-1', '1/2-1/2')
    """
    all_positions = []
    game_id = 0

    with open(pgn_file_path, "r", encoding="utf-8") as pgn:
        for i in tqdm(range(10_000)):
            game = chess.pgn.read_game(pgn)
            if game is None:
                break

            game_id += 1
            result = game.headers.get("Result", "*")
            board = game.board()

            move_number = 0
            for move in game.mainline_moves():
                board.push(move)
                move_number += 1
                position_data = {
                    "game_id": game_id,
                    "move_number": move_number,
                    "board_fen": board.fen(),
                    "move": move.uci(),
                    "result": result
                }
                all_positions.append(position_data)

    df = pd.DataFrame(all_positions)
    return df

In [11]:
pgn_path = r"C:\Users\forbe\Downloads\lichess_elite_2024-10\lichess_elite_2024-10.pgn"
df_positions = pgn_to_dataframe(pgn_path)

100%|██████████| 10000/10000 [00:35<00:00, 282.77it/s]


In [12]:
RESULT_MAPPING = {"1-0": 2, "0-1": 0, "1/2-1/2": 1}

def df_fen_to_binary_perspective(
    df: pd.DataFrame,
    fen_column: str = "board_fen",
    result_column: str = "result"
) -> pd.DataFrame:
    """
    Given a DataFrame that has columns:
      - fen_column (default 'board_fen'): FEN strings
      - result_column (default 'result'): game result
    Convert each FEN to a 780-dim feature vector from the side-to-move perspective.
    
    Adjust the result to reflect the perspective of the side to move.
    
    Returns a new DataFrame with:
      - 768 columns for piece placement
      - 4 columns for castling rights
      - 8 columns for en passant
      - 'result' column (adjusted to the side to move)
    """
    feature_list = []
    adjusted_results = []

    for fen, result in tqdm(zip(df[fen_column], df[result_column]), total=len(df)):
        # Extract features from the perspective of the side to move
        board = chess.Board(fen)
        side_to_move = board.turn  # True for White, False for Black
        
        # Mirror the board if Black is to move
        if not side_to_move:  # Black to move
            board = board.mirror()
            # Adjust the result: swap "1-0" with "0-1"
            if result == "1-0":
                adjusted_result = "0-1"
            elif result == "0-1":
                adjusted_result = "1-0"
            else:
                adjusted_result = result  # Draw remains the same
        else:
            adjusted_result = result  # White to move, no adjustment needed

        # Convert FEN to binary features
        feature_vec = fen_to_binary_features(board.fen())
        feature_list.append(feature_vec)
        adjusted_results.append(adjusted_result)

    # Convert the list of numpy arrays to a 2D array
    feature_array = np.vstack(feature_list)

    # Create column names
    num_features = 768 + 4 + 8  # = 780
    column_names = (
        [f"piece_placement_{i}" for i in range(768)]
        + [f"castling_{i}" for i in range(4)]
        + [f"en_passant_{i}" for i in range(8)]
    )

    # Build the DataFrame of features
    df_features = pd.DataFrame(feature_array, columns=column_names, index=df.index)

    # Add the adjusted result column, mapping it to numeric labels
    df_features[result_column] = adjusted_results
    df_features[f"{result_column}_label"] = df_features[result_column].map(RESULT_MAPPING)

    return df_features


In [13]:
top_positions = list(df_positions[df_positions.move_number == 4].board_fen.value_counts().index)

In [14]:
# chess.Board(top_positions[2])

In [15]:
df_features = df_fen_to_binary_perspective(df_positions, fen_column="board_fen")

100%|██████████| 866682/866682 [01:52<00:00, 7699.91it/s]


In [25]:
df_features

,piece_placement_0,piece_placement_1,piece_placement_2,piece_placement_3,piece_placement_4,piece_placement_5,piece_placement_6,piece_placement_7,piece_placement_8,piece_placement_9,...,en_passant_0,en_passant_1,en_passant_2,en_passant_3,en_passant_4,en_passant_5,en_passant_6,en_passant_7,result,result_label
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0-1,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1-0,2
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0-1,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1-0,2
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866677,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0-1,0
866678,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1-0,2
866679,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0-1,0
866680,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1-0,2


In [26]:
df_features_shuffled = df_features.sample(frac=1, random_state=42).reset_index(drop=True)

In [45]:
def train_mlp_classifier(df_features: pd.DataFrame):
    # Identify the 780 feature columns
    feature_cols = list(df_features.columns)[:-2]

    X = df_features[feature_cols].values
    y = df_features["result_label"]

    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.20,
        random_state=42,
        stratify=y
    )

    # Optional: scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Initialize MLP
    mlp = MLPClassifier(
        hidden_layer_sizes=(128),
        random_state=42,
        verbose=True
    )


    print("ready to train")

    # Fit (train) the model
    mlp.fit(X_train_scaled, y_train)

    # Evaluate
    accuracy = mlp.score(X_test_scaled, y_test)
    print(f"Test Accuracy: {accuracy:.3f}")

    y_pred = mlp.predict(X_test_scaled)
    print("Classification Report:")
    print(classification_report(y_test, y_pred, digits=3))

    # Return the trained classifier and scaler
    return mlp, scaler

In [46]:
mlp_model, feature_scaler = train_mlp_classifier(df_features_shuffled.iloc[:100_000])

ready to train
Iteration 1, loss = 1.05002165
Iteration 2, loss = 0.92320283
Iteration 3, loss = 0.87559618
Iteration 4, loss = 0.81990001
Iteration 5, loss = 0.76242064
Iteration 6, loss = 0.70154650
Iteration 7, loss = 0.64678759
Iteration 8, loss = 0.60105909
Iteration 9, loss = 0.56088261
Iteration 10, loss = 0.52792231
Iteration 11, loss = 0.49415083
Iteration 12, loss = 0.46972587
Iteration 13, loss = 0.44588177
Iteration 14, loss = 0.42651314
Iteration 15, loss = 0.40830987
Iteration 16, loss = 0.39210659
Iteration 17, loss = 0.37769185
Iteration 18, loss = 0.36799168
Iteration 19, loss = 0.35626512
Iteration 20, loss = 0.34698794
Iteration 21, loss = 0.33709602
Iteration 22, loss = 0.32594817
Iteration 23, loss = 0.31664537
Iteration 24, loss = 0.31160802
Iteration 25, loss = 0.30348406
Iteration 26, loss = 0.29816554
Iteration 27, loss = 0.28826454
Iteration 28, loss = 0.28623898
Iteration 29, loss = 0.28404258
Iteration 30, loss = 0.27664493
Iteration 31, loss = 0.27322502
It

In [47]:
import pickle

In [48]:
with open("pkl/model.pkl", "wb") as f:
    pickle.dump(mlp_model, f)

In [49]:
with open("pkl/scaler.pkl", "wb") as f:
    pickle.dump(feature_scaler, f)